# Intro to astropy.coordinates library

## Learning Goals
- will be able to do stuff

In this tutorial, we're going to investigate the area of the sky around the picturesque group of galaxies named "Hickson Compact Group 7". download an image and do something with it's coordinates Let's start by considering a field around 

### Imports

In [2]:
# Python standard-library
from urllib.parse import urlencode
from urllib.request import urlretrieve

# Third-party dependencies
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
import numpy as np
from IPython.display import Image

# Set up matplotlib and use a nicer set of plot parameters
from astropy.visualization import astropy_mpl_style
import matplotlib.pyplot as plt
plt.style.use(astropy_mpl_style)
%matplotlib inline

## Describing on-sky locations with `coordinates`

The `SkyCoord` class in the `astropy.coordinates` package is used to represent celestial coordinates. First, we'll make a SkyCoord object based on our object's name, "Hickson Compact Group 7", or "HCG 7" for short. Most astronomical object names can be found by [SESAME](http://cdsweb.u-strasbg.fr/cgi-bin/Sesame), a service which queries Simbad, NED, and VizieR and returns the object's type and its J2000 position. This service can be used via the `SkyCoord.from_name()` [class method](https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods). 

In [3]:
# initialize a SkyCood object named hcg7_center at the location of HCG 7
hcg7_center = SkyCoord.from_name('HCG 7')

<div class="alert alert-info">
Note that this requires an internet connection.  If you don't have one, execute this line instead:
</div>

In [4]:
# uncomment and run this line if you don't have an internet connection
#hcg7_center = SkyCoord(9.81625*u.deg, 0.88806*u.deg, frame='icrs')

In [13]:
hcg7_center

<SkyCoord (ICRS): (ra, dec) in deg
    ( 9.849602,  0.87817)>

We see that, according to SESAME, HCG 7 is located at ra = 9.849 deg and dec = 0.878 deg. 

This object we just created has various useful ways of accessing the information contained within it.  In particular, the ``ra`` and ``dec`` attributes are specialized [``Quantity``](http://docs.astropy.org/en/stable/units/index.html) objects (actually, a subclass called [``Angle``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html), which in turn is subclassed by [``Latitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html) and [``Longitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)).  These objects store angles and provide pretty representations of those angles, as well as some useful attributes to quickly convert to common angle units:

In [13]:
type(hcg7_center.ra), type(hcg7_center.dec)

(astropy.coordinates.angles.Longitude, astropy.coordinates.angles.Latitude)

In [14]:
hcg7_center.ra, hcg7_center.dec

(<Longitude 9.849602 deg>, <Latitude 0.87817 deg>)

 To do anything with this, we need to get an object that represents the coordinates of the center of this group.

  A `SkyCoord` can be created from angles as shown below.  It's also wise to explicitly specify the frame your coordinates are in, although this is not strictly necessary because the default is ICRS. 

(If you're not sure what ICRS is, it's basically safe to think of it as an approximation to an equatorial system at the J2000 equinox).

In [4]:
print(hcg7_center)

<SkyCoord (ICRS): (ra, dec) in deg
    ( 9.81625,  0.88806)>


SkyCoord will also accept string-formatted coordinates either as separate strings for ra/dec or a single string.  You'll have to give units, though, if they aren't part of the string itself.

In [5]:
SkyCoord('0h39m15.9s', '0d53m17.016s', frame='icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    ( 9.81625,  0.88806)>

(astropy.coordinates.angles.Longitude, astropy.coordinates.angles.Latitude)

In [8]:
hcg7_center.dec

<Latitude 0.87817 deg>

In [9]:
hcg7_center.ra

<Longitude 9.849602 deg>

In [10]:
hcg7_center.ra.hour

0.6566401333333335

Now that we have a `SkyCoord` object, we can try to use it to access data from the [Sloan Digitial Sky Survey](http://www.sdss.org/) (SDSS).  Let's start by trying to get a picture using the SDSS image cutout service to make sure HCG7 is in the SDSS footprint and has good image quality.

This requires an internet connection, but if it fails, don't worry: the file is included in the repository so you can just let it use the local file``'HCG7_SDSS_cutout.jpg'``, defined at the top of the cell.  

In [13]:
impix = 1024
imsize = 12*u.arcmin
cutoutbaseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
query_string = urlencode(dict(ra=hcg7_center.ra.deg, 
                              dec=hcg7_center.dec.deg, 
                              width=impix, height=impix, 
                              scale=imsize.to(u.arcsec).value/impix))
url = cutoutbaseurl + '?' + query_string

# this downloads the image to your disk
urlretrieve(url, 'HCG7_SDSS_cutout.jpg')

('HCG7_SDSS_cutout.jpg', <http.client.HTTPMessage at 0x112af7400>)

In [ ]:
Image('HCG7_SDSS_cutout.jpg')

Very pretty!

### Exercise 1

Create a `SkyCoord` of some other astronomical object you find interesting. Using only a single method/function call, get a string with the RA/Dec in the form 'HH:MM:SS.S DD:MM:SS.S'.  Check your answer against an academic paper or some web site like [SIMBAD](http://simbad.u-strasbg.fr/simbad/) that will show you sexigesimal coordinates for the object.

(Hint: `SkyCoord.to_string()` might be worth reading up on)

In [14]:
coord = SkyCoord.from_name('M13')

Now get an image of that object from the Digitized Sky Survey and download it and/or show it in the notebook. Bonus points if you figure out the (one-line) trick to get it to display in the notebook *without* ever downloading the file yourself.

(Hint: STScI has an easy-to-access [copy of the DSS](https://archive.stsci.edu/dss/).  The pattern to follow for the web URL is ``http://archive.stsci.edu/cgi-bin/dss_search?f=GIF&ra=RA&dec=DEC``)

## Passing in an array of coordinates

If we have many coordinates(say from a data table) we can pass them all into astropy coordinates. Let's first read in a table of coordinates from 2MASS using astropy Table:

In [21]:
hcg7_2mass = Table.read('HCG7_2MASS.tbl', format='ascii')

We can look at the data table below:

In [24]:
hcg7_2mass

designation,ra,dec,r_k20fe,j_m_k20fe,j_msig_k20fe,j_flg_k20fe,h_m_k20fe,h_msig_k20fe,h_flg_k20fe,k_m_k20fe,k_msig_k20fe,k_flg_k20fe,k_ba,k_phi,sup_ba,sup_phi,r_ext,j_m_ext,j_msig_ext,h_m_ext,h_msig_ext,k_m_ext,k_msig_ext,cc_flg,dist,angle
,deg,deg,arcsec,mag,mag,,mag,mag,,mag,mag,,,deg,,deg,arcsec,mag,mag,mag,mag,mag,mag,,arcsec,deg
str16,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64
00402069+0052508,10.086218,0.880798,9.4,13.835,0.068,0,13.01,0.086,0,12.588,0.089,0,0.8,70,0.82,35,18.62,13.632,0.088,12.744,0.104,12.398,0.105,0,972.120611,91.538952
00395984+0103545,9.99935,1.06514,12.9,12.925,0.035,0,12.183,0.042,0,11.89,0.067,0,0.8,35,0.7,40,35.9,12.469,0.048,11.91,0.066,11.522,0.087,0,916.927636,45.951861
00401849+0049448,10.077062,0.82913,6.0,14.918,0.086,0,14.113,0.107,0,13.714,0.103,0,0.6,-15,1.0,90,11.35,14.631,0.121,13.953,0.169,13.525,0.161,0,962.489231,102.73149
00395277+0057124,9.969907,0.953472,5.3,14.702,0.049,0,14.248,0.069,0,13.899,0.095,0,0.6,-60,0.44,-50,10.59,14.62,0.144,14.15,0.296,13.73,0.2,0,601.136444,66.93659
00401864+0047245,10.077704,0.790143,7.6,15.585,0.134,1,15.003,0.18,1,14.049,0.142,1,0.5,30,0.46,30,14.48,14.977,0.138,14.855,0.303,13.653,0.18,0,1004.982128,110.53147
00393485+0051355,9.895219,0.859882,39.3,11.415,0.031,3,10.755,0.044,3,10.514,0.068,3,0.6,-30,0.7,-60,92.29,11.415,0.018,10.155,0.054,9.976,0.085,0,301.813395,109.639102
00392964+0103495,9.873526,1.063769,10.9,14.463,0.065,0,13.618,0.067,0,13.258,0.091,0,0.4,55,0.28,60,20.35,14.2,0.086,13.363,0.091,13.101,0.133,0,665.301415,18.051526
00403343+0049079,10.139293,0.818865,5.0,15.484,0.15,0,--,--,--,13.97,0.137,0,1.0,90,1.0,90,10.05,15.035,0.183,14.725,0.0,13.654,0.189,0,1189.207905,102.088788


If we wanted to make an list of coordinates we could use a for-loop:

In [42]:
coordinates_2mass = []  # start with an empty list
#for i in range(len(hcg7_2mass['ra'])):
    #ra = hcg7_2mass['ra'][i]
    #dec = hcg7_2mass['dec'][i]
    
for i, (ra, dec) in enumerate(zip(hcg7_2mass['ra'], hcg7_2mass['dec'])):
    coordinates_2mass.append(SkyCoord(ra*u.deg, dec*u.deg))  # add a new coordinate
    
print(coordinates_2mass)

[<SkyCoord (ICRS): (ra, dec) in deg
    ( 10.086218,  0.880798)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.99935,  1.06514)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 10.077062,  0.82913)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.969907,  0.953472)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 10.077704,  0.790143)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.895219,  0.859882)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.873526,  1.063769)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 10.139293,  0.818865)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.888305,  0.597381)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.959649,  0.595693)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.928026,  0.627663)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.912468,  0.642384)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 10.028815,  0.683197)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.824936,  0.699687)>, <SkyCoord (ICRS): (ra, dec) in deg
    ( 9.699858,  0.582578)>, <SkyCoord (ICRS): (ra, dec) in deg
  

We can make this even easier though! We can pass SkyCoord arrays of coordinates and get an array back:

In [40]:
# this table already has units associated with the columns so we do not need to specify them
coordinates_2mass = SkyCoord(hcg7_2mass['ra'], hcg7_2mass['dec'])
print(coordinates_2mass)

<SkyCoord (ICRS): (ra, dec) in deg
    [( 10.086218,  0.880798), (  9.99935 ,  1.06514 ),
     ( 10.077062,  0.82913 ), (  9.969907,  0.953472),
     ( 10.077704,  0.790143), (  9.895219,  0.859882),
     (  9.873526,  1.063769), ( 10.139293,  0.818865),
     (  9.888305,  0.597381), (  9.959649,  0.595693),
     (  9.928026,  0.627663), (  9.912468,  0.642384),
     ( 10.028815,  0.683197), (  9.824936,  0.699687),
     (  9.699858,  0.582578), (  9.766345,  0.849419),
     (  9.805797,  0.864135), (  9.824418,  0.912743),
     (  9.744971,  0.957478), (  9.828303,  0.891909),
     (  9.80055 ,  1.044691), (  9.666268,  1.078968),
     (  9.704872,  0.839244)]>
